In [132]:
import twitter
import re
import json
import pandas as pd

def left(s, amount = 1, substring = ""):
    if (substring == ""):
        return s[:amount]
    else:
        if (len(substring) > amount):
            substring = substring[:amount]
        return substring + s[:-amount]

def right(s, amount = 1, substring = ""):
    if (substring == ""):
        return s[-amount:]
    else:
        if (len(substring) > amount):
            substring = substring[:amount]
        return s[:-amount] + substring
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output


CONSUMER_KEY ="wL8xOLpaozc5igQgSpc5SmWRu"
CONSUMER_SECRET ="3pUVWDURtNojYGITOtAVZkLm1t4ZHtvxdZSBWiTYgWxa1E5KoH"
OAUTH_TOKEN = "921778257032351744-9BtaFSbyYQ7SAVkvAdQxo3nz2XIUmso"
OAUTH_TOKEN_SECRET = "0eXbrJWeZiTJCOcvK7vQ6HeedszbIo17MMpgGQBAvskHm"

auth = twitter.oauth.OAuth (OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET)
twitter_api= twitter.Twitter(auth=auth)
search_results = twitter_api.search.tweets(q='#soccer',count='1000000',lang='en')
statuses = search_results['statuses']

for _ in range(100):
    print ("Length of statuses", len(statuses))
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError:   
        break
   # Create a dictionary from next_results
    kwargs=dict( [kv.split('=') for kv in next_results[1:].split("&") ])

    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']

#print (statuses)
df = pd.DataFrame(columns=['created_at','urls','hashtags','text'])
i=0
for status in statuses:
    hashtags=[]
    urls=[]
    for hashtag in status['entities']['hashtags']:
        h = hashtag['text'].lower()
        hashtags.append(h) 
    for url in status['entities']['urls']:
        urls.append(url['url']) 
    texts =  status['text'].lower()
    texts =  re.sub(r'https?:\/\/.*[\r\n]*', '', texts, flags=re.MULTILINE)
    texts = re.sub(r'[?|$|.|&|[|!|%|\-|*|;|#|"|\n|,|@|(|)]',r'',texts)
    texts = re.sub(r'[0|1|2|3|4|5|6|7|8|9|:]',r'',texts)       

    created_ats = status['created_at']
    created_ats = pd.to_datetime(created_ats)
    df.loc[i]=[created_ats,urls,hashtags,texts]
    i+=1

texts = df['text']    
#print (df.head())
for counter, t in enumerate(texts):
    if t.startswith("rt @"):
        pos= t.find(": ")
        texts[counter] = right(t, len(t) - (pos+2))

texts= remove_duplicates(texts)

Length of statuses 100
Length of statuses 200
Length of statuses 200


In [134]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from nltk import bigrams 

count_all = Counter()  #counter for all the words
terms_all = [] #list of all the tokens in tweets
terms_bigram = []
stop = stopwords.words('english') +['rt']  #removing retweet 'rt'
for t in texts:
    tokens = [i for i in word_tokenize(t.lower()) if i not in stop]
    count_all.update(tokens)
    terms_all.extend(tokens)

    
print (count_all.most_common(10))
terms_all = list(set(terms_all))
#print(terms_all)

[('soccer', 128), ('game', 17), ('⚽️', 15), ("'s", 13), ('west', 13), ('southampton', 13), ('win', 12), ('streams', 11), ('boufal', 11), ("'", 10)]


In [122]:
import nltk
import re
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

stemmer = SnowballStemmer("english")

#take comments from dataframe
textDF = df['text']

#token given text and find their stems
def tokenize(text):
    textTokens=[]
    #for userComments in commentsDf:
    textTokens = (nltk.word_tokenize(text))
    #filter out punctuations and numeric tokens
    return textTokens

#create stem dictionary for all the commments
totalVocabTokens = []
for i in textDF:
    totalVocabTokens.extend(nltk.word_tokenize(i))

print (totalVocabTokens)
#making Term Frequency-Inverse document Frequency matrix model
tfidf_vectorizer = TfidfVectorizer(max_df=0.8,
                                 min_df=0, stop_words='english',
                                 use_idf=True,tokenizer=tokenize, ngram_range=(1,3))  #creating features by taking 3 words at a time

tfidf_matrix = tfidf_vectorizer.fit_transform(textDF)
print (tfidf_matrix.shape)
terms = tfidf_vectorizer.get_feature_names()
#terms are list of features in the matrix
#print (terms)



['after', 'game', 'snack', 'win', 'how', 'cute', 'is', 'this', 'soccer', 'cookie', 'soccercookies', 'soccermomx', 'georgetown…', 'great', 'team', 'moments', 'chicago', 'fire', 'mls', 'soccer', 'at', 'enthusiasts', 'some', 'action', 'off', 'a', 'corner', 'in', 'today', "'s", 'soccer', 'matchup', 'gburgsoccer', 'came', 'away', 'with', 'a', 'victory', 'over', 'wheaton', 'leagueofirelandpremierdivision', 'sligo', 'rovers', 'v', 'derry', 'city', 'buydirectsports', 'premium', 'play', 'winner', 'army', 'ml', 'on', 'to', 'nfl', 'mlb', 'soccer', 'cfb', 'nba', 'amp', ';', 'cbb', 'winners…', 'scored', 'three', 'goals', 'and', 'a', 'new', 'team', 'bought', 'me', 'i', 'got', 'transferred', 'soccer⚽⚽', 'this', 'young', 'man', 'scored', 'not', 'one', 'but', 'two', 'goals', 'today', 'littleman', 'soccer', 'isports', 'ggkcraftypad', '…', 'rt', 'hawksbet', 'free', 'picks', 'retweet', 'and', 'ill', 'send', 'you', 'the', 'picksfollow', 'mesportsbetting', 'freepicks', 'mlb', 'soccer', 'nfl', 'nhl', 'soccer

In [123]:
from sklearn.cluster import KMeans,DBSCAN

num_clusters = 5
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

db = DBSCAN(eps=0.4,min_samples=6)
db.fit(tfidf_matrix)
clustersDB = db.labels_.tolist()
clusters = km.labels_.tolist()
#clusters is a list corresponding to 1338 rows in df
#print (clusters)
#add clusters to dataframe
df['clusters']  =clustersDB
#print (df.head())
#show the distribution of clusters
#print (df['clusters'].value_counts())

order_centroids = km.cluster_centers_.argsort()[:,::-1]
print (order_centroids)

for centres in order_centroids:
    topFeatures=[]
    for words in centres[:6]:
        topFeatures.append(terms[words])
    print (topFeatures)

[[ 143  142   45 ..., 2145 2144    0]
 [2368  957 2106 ...,  709  708 3297]
 [2594 3072  378 ..., 2145 2144 1648]
 [2106 2456 1859 ..., 2194 2193    0]
 [3232 2527  488 ..., 2123 2122    0]]
['amp ;', 'amp', ';', 'bordeaux', 'team', 'soccer']
['soccer', 'game', 'rt', 'soccer soccer', 'football', '’']
['southampton', 'west brom', 'brom', 'west', 'southampton west brom', 'southampton west']
['rt', 'soccer nfl nhl', 'picksfollow mesportsbetting', 'ill send', 'ill send picksfollow', 'retweet ill send']
['⚽️', 'soccer ⚽️', 'city', 'soccer', 'palace', 'man']


In [124]:
from sklearn.manifold import MDS
#dimensionality reduction for the TF-IDF matrix (or df equally)
MDS()

# convert two components as we're plotting points in a two-dimensional plane

# we will also specify `random_state` so the plot is reproducible.
mds = MDS(n_components=2, random_state=1)

#toarray() converts sparse array to dense numpy array
pos = mds.fit_transform(tfidf_matrix.toarray())  # shape (n_components, n_samples)

#store the dimensions in xs, ys
xs, ys = pos[:, 0], pos[:, 1]


In [125]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, Select,LabelSet

def get_colors(clusters):
    colors=[]
    for i in clusters:
        if i==-1:
            colors.append('violet')
        if i==0:
            colors.append('red')
        if i==1:
            colors.append('blue')
        if i==2:
            colors.append('green')
        if i==3:
            colors.append('brown')
        if i==4:
            colors.append('yellow')
        if i==5:
            colors.append('pink')
        if i==6:
            colors.append('orange')
    return colors

colors = get_colors(clusters)



output_notebook()
'''
a=figure(x_range =list(set(list(df['user']))),plot_height=500, plot_width=900, title='clustering groups based on edits')
data = ColumnDataSource(data=dict(x=df['user'],y=df['timestamp'],colors = colors,label=df['tags']))
a.square('x','y',fill_color='colors',line_color='colors',source=data,size=10)
#users are label
labels = LabelSet(x='x', y='y', text='label', level='glyph',
                  x_offset=5, y_offset=5, source=data, render_mode='canvas')
#a.add_layout(labels)
show(a)
'''
#print(data.column_names)
b=figure( plot_height=500, plot_width=900, title='clustering groups based on edits')
data1 = ColumnDataSource(data=dict(x=xs,y=ys,colors = colors,label=df['text']))
b.square('x','y',fill_color='colors',line_color='colors',source=data1,size=10)
#user are labels
labels = LabelSet(x='x', y='y', text='label', level='glyph',
                  x_offset=5, y_offset=5, source=data1,render_mode='canvas')
b.add_layout(labels)
show(b)

Loading BokehJS ...